In [1]:
import os
import json
from keras.preprocessing.text import Tokenizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
from nltk.tokenize import RegexpTokenizer
import gensim

/Users/shengyuan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(dir,round,args):
    data = []
    data_args = []
    for i in range(1,6):
        with open(os.path.join(dir,'r%d/%d.json'%(round,i)),'r') as d:
            tem_data = json.load(d)
        data.extend(tem_data['result']['docs'])
    for i in range(len(data)):
        data_args.append(data[i][args])
        data_args = list(set(data_args))
    return data_args


In [3]:
# load the data
dir='/Users/shengyuan/Desktop/Study/CAPSTONE/RoyalCommission' #data fold path
args = 'content'  #data type
round=4
max_words=2000
data = load_data(dir,round,args)
# Number of corpus
l=len(data)

In [4]:
l

620

In [5]:
from gensim.models import Doc2Vec

In [6]:
data[0]

'The Financial Services Royal Commission is wrapping up its Darwin hearings that have focussed on the treatment of rural and Indigenous Australians. \n\nAmong the revelations has been details of two funeral insurance companies that may have broken the law in targeting Aboriginal and Torres Strait Islander customers.'

In [7]:
#Data cleaning
def text_cleaning(text):
    myText = text.lower()    
    # clean and tokenize document string
    document_content = myText.split()    
    word_list = []
    for i in document_content:
        x = 0
        if (('http' not in i) and ('@' not in i) and ('<.*?>' not in i) and i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list 

In [8]:
#Data Pre-processing
def preprocessing(text):    
    # remove numbers
    number_tokens = [re.sub(r'[\d]', ' ', i) for i in text]
    number_tokens = ' '.join(number_tokens).split()
     # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in number_tokens]
    # remove empty
    length_tokens = [i for i in stemmed_tokens if len(i) > 1]
    return length_tokens

In [9]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content = []
texts = []
j=0
k=0
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
p_stemmer = PorterStemmer()
for em in data:           
    #Data cleaning
    clean_content = text_cleaning(em)
    
    #Pre-processing
    processed_content = preprocessing(clean_content)
    
    # add tokens to list
    if processed_content:
        all_content.append(LabeledSentence1(processed_content,[j]))
        j+=1
        
    k+=1

print("Number of article: ", k)
print("Number of non-empty article vectors: ", j)

Number of article:  620
Number of non-empty article vectors:  620


In [10]:
d2v_model = Doc2Vec(all_content, vector_size = 2000, window = 10, min_count = 500, workers=7, dm = 1, 
                alpha=0.025, min_alpha=0.001)
d2v_model.train(all_content, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)

In [11]:
print (d2v_model.docvecs.most_similar(1))

[(575, 0.9913215637207031), (476, 0.9911184310913086), (608, 0.9910399913787842), (522, 0.9909705519676208), (553, 0.9898577332496643), (567, 0.9895543456077576), (218, 0.9895424842834473), (602, 0.9893099665641785), (73, 0.9892160892486572), (245, 0.9892145395278931)]


/Users/shengyuan/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [12]:
all_content[2]

TaggedDocument(words=['coal', 'kid', 'energi', 'minist', 'josh', 'frydenberg', 'reportedli', 'draft', 'plan', 'bring', 'extra', 'coal', 'ga', 'power', 'attempt', 'squar', 'nation', 'energi', 'guarante', 'coalit', 'australian', 'report', 'coalit', 'mp', 'brief', 'plan', 'nation', 'leader', 'deputi', 'prime', 'minist', 'michael', 'mccormack', 'look', 'lobbi', 'pm', 'malcolm', 'turnbul', 'facilit', 'new', 'coal', 'power', 'relat', 'ndevr', 'environment', 'found', 'australian', 'carbon', 'emiss', 'broke', 'record', 'past', 'act', 'continu', 'threaten', 'block', 'neg', 'emiss', 'target', 'set', 'larg', 'met', 'inquiri', 'nsw', 'doc', 'new', 'south', 'wale', 'health', 'depart', 'launch', 'major', 'inquiri', 'gynaecologist', 'period', 'reportedli', 'mutil', 'perform', 'unnecessari', 'surgeri', 'dozen', 'includ', 'one', 'die', 'inappropri', 'accord', 'health', 'author', 'set', 'independ', 'special', 'counsel', 'inquiri', 'along', 'dedic', 'telephon', 'line', 'nsw', 'hospit', 'dr', 'emil', 'sha

In [13]:
print(d2v_model.docvecs.vectors_docs.shape)

(620, 2000)


In [14]:
doc_vector=d2v_model.docvecs.vectors_docs;

In [ ]:
def k_means(num_clusters,data):
    km = KMeans(n_clusters=num_clusters)
    km.fit(data)
    clusters = km.labels_.tolist()
    score=silhouette_score(data,km.labels_,metric='euclidean')
    
    return clusters, km.inertia_,score

In [ ]:
SSE_list=list()
Scores = list()
clusters_range = np.arange(2,300,10)

# SSE(sum of the squared errors)
# Silhouette Coefficient
for i in clusters_range:
    clusters,SSE,score=k_means(i,doc_vector)
    Scores.append(score)
    SSE_list.append(SSE)
plt.title( 'K Number Vs SSE')
plt.xlabel('K Number')
plt.ylabel('SSE')
plt.plot(clusters_range,SSE_list,'o-')
plt.show()


plt.title( 'K Number Vs Silhouette Coefficient')
plt.xlabel('K Number')
plt.ylabel('Silhouette Coefficient')
plt.plot(clusters_range,Scores,'o-')
plt.show()

In [ ]:
#set up colors per clusters using a dictclusters,SSE,score=k_means(i,PCA_data)
num_clusters=10
clusters,SSE,score=k_means(num_clusters,doc_vector)
cluster_colors=[]
cluster_names =[]

cluster_colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(num_clusters)]


print(cluster_colors)
for i in range(num_clusters):
    name="Cluster"+ str (i+1)
    cluster_names.append(name)
print(cluster_names)
print(clusters)

In [ ]:
# calculate the similarity

dist = 1 - cosine_similarity(doc_vector)

# For plotting purpose
pca = PCA(n_components=2, copy=True)
pos = pca.fit_transform(dist) # shape (n_components, n_samples)
xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
#some ipython magic to show the matplotlib plots inline
%matplotlib inline 

#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=np.arange(1,pos.shape[0]+1))) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.loc[df.index[i],'x'], df.loc[df.index[i],'y'], df.loc[df.index[i],'title'], size=7)  
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)